# Case Study on Online Movie Rental Company

This is based on a case study about an online movie rental company with a database about customer information, movie ratings, background information on actors and more. 

Applied SQL queries to study for example customer preferences, customer engagement, and sales development and also used SQL extensions for online analytical processing (OLAP), which makes it easier to obtain key insights from multidimensional aggregated data.

# Introduction

Worked with a Postgres database from a fictional movie rental company called MovieNow. 

MovieNow offers an online platform for streaming movies. Customers can rent a movie for 24 hours. For all movies, the company stores additional information such as the genre or the main actors. MovieNow also stores information about customers and movie ratings.


# Schema

The schema contains 5 tables : customers, movies, renting, actors, actsin.

Customers (customer_id, name, country, gender, date of birth, date_account_start) 
 
Movies (movie_id, title, genre, runtime, year_of_releas, renting_price) 

Renting (renting_id, customer_id, movie_id, rating, date_renting) 

Actors (actor_id, name, year_of_birth, nationality, gender) 

Actsin (actsin_id, actor_id, movie_id) 

# KPI's

KPI's help company define and monitor success. For the MovieNow Company KPI's are:

Revenue - the total number of rentals.

Customer satisfaction - average rating of all movies.

Customer Engagement - number of active customers.

# Data Exploration

In [1]:
SELECT *
FROM movies.actors
LIMIT 5;

,actor_id,name,year_of_birth,nationality,gender
0,1,Abbie Cornish,1982,Australia,female
1,2,Adam Sandler,1966,USA,male
2,3,Al Pacino,1940,USA,male
3,4,Amy Adams,1974,USA,female
4,5,Andrea Riseborough,1981,British,female


Let's see how many actors are from 'USA'

In [10]:
SELECT COUNT(*)
FROM movies.actors
WHERE nationality = 'USA';

,count
0,91


There are 91 actors from USA.

In [6]:
SELECT *
FROM movies.customers
LIMIT 5;

,customer_id,name,country,gender,date_of_birth,date_account_start
0,2,Wolfgang Ackermann,Austria,male,1971-11-17 00:00:00+00:00,2018-10-15 00:00:00+00:00
1,3,Daniela Herzog,Austria,female,1974-08-07 00:00:00+00:00,2019-02-14 00:00:00+00:00
2,4,Julia Jung,Austria,female,1991-01-04 00:00:00+00:00,2017-11-22 00:00:00+00:00
3,5,Juliane Kirsch,Austria,female,1977-03-01 00:00:00+00:00,2018-12-16 00:00:00+00:00
4,6,Rowanne Couperus,Belgium,female,1994-04-05 00:00:00+00:00,2018-08-26 00:00:00+00:00


Lets see the number of countries where MovieNow has customers.

In [11]:
SELECT COUNT( DISTINCT country)
FROM movies.customers;

,count
0,11


MovieNow has customers in 11 countries 

In [3]:
SELECT *
FROM movies.movies
LIMIT 5;

,movie_id,title,genre,runtime,year_of_release,renting_price
0,1,One Night at McCool's,Comedy,93,2001,2.09
1,2,Swordfish,Drama,99,2001,2.19
2,3,What Women Want,Comedy,127,2001,2.59
3,4,Training Day,Drama,122,2001,1.79
4,5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


Let's see the distinct available genres 

In [17]:
SELECT DISTINCT genre
FROM movies.movies;

,genre
0,Animation
1,Other
2,Drama
3,Art House & International
4,Mystery & Suspense
5,Comedy
6,Action & Adventure
7,Science Fiction & Fantasy


There are 8 distinct genres in the database.

In [4]:
SELECT *
FROM movies.actsin
LIMIT 5;

,actsin_id,movie_id,actor_id
0,1,37,1
1,2,56,2
2,3,10,3
3,4,14,3
4,5,29,3


In [5]:
SELECT *
FROM movies.renting
LIMIT 5;

,renting_id,customer_id,movie_id,rating,date_renting
0,1,41,8,NaN,2018-10-09 00:00:00+00:00
1,2,10,29,10.0,2017-03-01 00:00:00+00:00
2,3,108,45,4.0,2018-06-08 00:00:00+00:00
3,4,39,66,8.0,2018-10-22 00:00:00+00:00
4,5,104,15,7.0,2019-03-18 00:00:00+00:00


Lets calc. the avg. rating per movie.

In [6]:
SELECT movie_id, rating
FROM movies.renting;

,movie_id,rating
0,8,NaN
1,29,10.0
2,45,4.0
3,66,8.0
4,15,7.0
...,...,...
573,14,NaN
574,31,10.0
575,25,NaN
576,65,9.0


Looks like the column rating in the renting table has a lot of null values since not every customer gives a rating after watching a movie.

# Data Analysis 

## 1. First account for each country

In [1]:
-- Finding when the first customer accounts were created for each country

SELECT country, MIN(date_account_start) AS first_account
FROM movies.customers
GROUP BY country
ORDER BY first_account;


,country,first_account
0,France,2017-01-13 00:00:00+00:00
1,Hungary,2017-01-18 00:00:00+00:00
2,Belgium,2017-01-28 00:00:00+00:00
3,Slovenia,2017-01-31 00:00:00+00:00
4,Spain,2017-02-14 00:00:00+00:00
5,Italy,2017-02-28 00:00:00+00:00
6,Poland,2017-03-03 00:00:00+00:00
7,Great Britan,2017-03-31 00:00:00+00:00
8,Denmark,2017-04-30 00:00:00+00:00
9,USA,2017-09-13 00:00:00+00:00


We can see that the first customer account was created in France.

## 2. Average movie ratings

In [2]:
-- lets see for each movie the average rating, the number of ratings and the number of views

SELECT movie_id, 
       AVG(rating) AS avg_rating,
       COUNT(rating) AS number_ratings,
       COUNT(*) AS number_renting
FROM movies.renting
GROUP BY movie_id
ORDER BY avg_rating DESC; 

,movie_id,avg_rating,number_ratings,number_renting
0,11,NaN,0,6
1,48,10.000000,2,5
2,5,9.750000,4,8
3,42,9.600000,5,10
4,63,9.333333,6,11
...,...,...,...,...
66,14,6.500000,6,11
67,68,6.333333,3,7
68,59,6.000000,2,5
69,43,6.000000,1,4


The average rating for movie_id 11  is null because all rating values are null for that movie id.

## 3. Average rating per customer

In [1]:
-- We see the avg. movie ratings given by each customer, number of ratings, number of rentals and analyse customers having rentals more than 7.

SELECT customer_id, 
       AVG(rating) AS avg_rating,
       COUNT(rating) AS number_ratings,
       COUNT(*) AS number_renting
FROM movies.renting
GROUP BY customer_id
HAVING COUNT(*)>7
ORDER BY avg_rating; 

,customer_id,avg_rating,number_ratings,number_renting
0,104,6.250000,4,8
1,28,6.714286,7,11
2,111,7.000000,3,10
3,113,7.000000,7,15
4,25,7.200000,5,10
5,21,7.333333,6,14
6,92,7.571429,7,11
7,49,7.625000,8,13
8,35,7.666667,6,9
9,52,7.875000,8,9


We see the customer number 104 gave the lowest average ratings for 4 movies.

## 4. Joining Renting and Customers.

In [5]:
-- Adding customer info to renting table to find the avg. ratings of customers from France

SELECT AVG(rating) 
FROM movies.renting AS r
LEFT JOIN movies.customers AS c
ON r.customer_id = c.customer_id
WHERE c.country='France';

,avg
0,7.771429


The average rating for customers from France is 7.77.

## 5. Aggregating revenue, rentals and active customers.

In [6]:
-- Reporting the KPI's for the performance of company in 2018.

SELECT 
	SUM(m.renting_price) AS revenue, 
	COUNT(*) AS number_rentals, 
	COUNT(DISTINCT r.customer_id) AS number_active_customers
FROM movies.renting AS r
LEFT JOIN movies.movies AS m
ON r.movie_id = m.movie_id
WHERE date_renting BETWEEN '2018-01-01' AND '2018-12-31' ;

,revenue,number_rentals,number_active_customers
0,658.02,298,93


We calculated a turnover of 658.02 and found the number of rentals to be 298 and the number of active users to be 93 in 2018.

## 6. Movies and Actors

In [7]:
--Listing which actors played in which movie

SELECT m.title, a.name
FROM movies.actsin AS ai
LEFT JOIN movies.movies AS m
ON m.movie_id = ai.movie_id
LEFT JOIN movies.actors AS a
ON a.actor_id = ai.actor_id

,title,name
0,Candy,Abbie Cornish
1,Jack and Jill,Adam Sandler
2,Simone,Al Pacino
3,The Recruit,Al Pacino
4,Two for the Money,Al Pacino
...,...,...
208,The Help,Viola Davis
209,Ghost Rider: Spirit of Vengeance,Violante Placido
210,Rio 2,will.i.am
211,The Sessions,William H. Macy


## 7. Income from movies.

In [1]:
SELECT rm.title,
SUM(rm.renting_price) AS income_movie
FROM
       (SELECT m.title,  
               m.renting_price
       FROM movies.renting AS r
       LEFT JOIN movies.movies AS m
       ON r.movie_id=m.movie_id) AS rm
GROUP BY rm.title
ORDER BY income_movie DESC; 

,title,income_movie
0,Bridget Jones - The Edge of Reason,37.57
1,Fair Game,34.68
2,The Kingdom,31.35
3,Two for the Money,30.69
4,Django Unchained,29.59
...,...,...
66,Harry Potter and the Chamber of Secrets,8.95
67,Jack and Jill,8.36
68,The Invasion,8.36
69,The Human Stain,7.96


Bridget Jones movies has the highest income of 37.57

## 8.Age of actors from the USA

In [2]:
-- Explore the age of American actor and actress.

SELECT a.gender,MIN(a.year_of_birth), MAX(a.year_of_birth)
FROM (SELECT * FROM movies.actors  WHERE nationality = 'USA') as a
GROUP BY a.gender;

,gender,min,max
0,female,1945,1993
1,male,1930,1992


MovieNow actors' record the oldest actor was born in 1930 and the oldest actress in 1945.

## 9. Identifying favorite movies for a group of customers

In [4]:
-- Lets take the case for a specific group of customers: for all customers born in the 70s.

SELECT m.title, 
COUNT(*),
AVG(r.rating)
FROM movies.renting AS r
LEFT JOIN movies.customers AS c
ON c.customer_id = r.customer_id
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
WHERE c.date_of_birth BETWEEN '1970-01-01' AND '1979-12-31'
GROUP BY m.title
HAVING COUNT(*)>1 -- Remove movies with only one rental
ORDER BY AVG(r.rating) DESC; -- Order with highest rating first

,title,count,avg
0,Showtime,5,NaN
1,Harry Potter and the Deathly Hallows – Part 2,2,NaN
2,Waking Up in Reno,2,NaN
3,Ray,2,NaN
4,Django Unchained,4,10.000000
5,The Fighter,4,10.000000
6,One Night at McCool's,2,10.000000
7,No Country for Old Men,3,10.000000
8,The Fellowship of the Ring,2,10.000000
9,I'm Not There,2,10.000000


'Django Unchained' is a movie with one of the best ratings among people born in the 70s.

## 10. Identify favorite actors for Spain

In [5]:
SELECT a.name,  c.gender,
       COUNT(*) AS number_views, 
       AVG(r.rating) AS avg_rating
FROM movies.renting as r
LEFT JOIN movies.customers AS c
ON r.customer_id = c.customer_id
LEFT JOIN movies.actsin as ai
ON r.movie_id = ai.movie_id
LEFT JOIN movies.actors as a
ON ai.actor_id = a.actor_id
WHERE c.country = 'Spain' -- Select only customers from Spain
GROUP BY a.name, c.gender
HAVING AVG(r.rating) IS NOT NULL 
  AND COUNT(*) > 5 
ORDER BY avg_rating DESC, number_views DESC;

,name,gender,number_views,avg_rating
0,Catherine Keener,female,6,8.0
1,Emma Watson,male,7,7.6
2,Daniel Radcliffe,male,7,7.6
3,Rupert Grint,male,7,7.6


Catherine Keener is the favorite actress among female Spain customers.

Male customers from Spain like the actors from Harry Potter best: Emma Watson, Daniel Radcliffe and Rupert Grint.

## 11. KPI's per country

In [6]:
-- Reporting KPI's for each country from begining of year 2019.

SELECT 
	 c.country ,               
	COUNT(*) AS number_renting, 
	AVG(r.rating) AS average_rating, 
	SUM(m.renting_price) AS revenue        
FROM movies.renting AS r
LEFT JOIN movies.customers AS c
ON c.customer_id = r.customer_id
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
WHERE date_renting >= '2019-01-01'
GROUP BY c.country;


,country,number_renting,average_rating,revenue
0,None,1,10.000000,1.79
1,Spain,26,8.076923,57.94
2,Great Britan,9,7.200000,17.91
3,Austria,4,6.000000,8.16
4,Italy,30,8.235294,69.90
5,Poland,21,8.125000,49.29
6,Slovenia,14,8.000000,30.26
7,Hungary,11,7.166667,25.89
8,Denmark,7,7.714286,14.63
9,Belgium,7,9.000000,13.13


We calculated a total revenue of 57.94 for Spain, with 26 movie rentals and an average rating of 8.1.

## 12. Often rented movies.

In [1]:
-- List all movies with more than 5 views

SELECT *
FROM movies.movies
WHERE movie_id IN  
(SELECT movie_id
	FROM movies.renting
	GROUP BY movie_id
	HAVING COUNT(*) > 5)

,movie_id,title,genre,runtime,year_of_release,renting_price
0,1,One Night at McCool's,Comedy,93,2001,2.09
1,2,Swordfish,Drama,99,2001,2.19
2,3,What Women Want,Comedy,127,2001,2.59
3,4,Training Day,Drama,122,2001,1.79
4,5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59
5,6,Harry Potter and the Philosopher's Stone,Science Fiction & Fantasy,152,2001,2.69
6,7,The Royal Tenenbaums,Comedy,110,2002,1.89
7,10,Simone,Drama,117,2002,2.69
8,11,Showtime,Comedy,95,2002,1.79
9,12,The Two Towers,Science Fiction & Fantasy,179,2002,2.39


58 movies are often rented in which 7 belong to Harry Potter Series

## 13. Frequent Customers

In [2]:
-- List all info of customers who rented more than 10 movies.

SELECT *
FROM movies.customers
WHERE customer_id IN           
	(SELECT customer_id
	FROM movies.renting
	GROUP BY customer_id
	HAVING COUNT(*)>10);

,customer_id,name,country,gender,date_of_birth,date_account_start
0,21,Avelaine Corbeil,France,female,1986-03-17 00:00:00+00:00,2017-06-11 00:00:00+00:00
1,28,Sidney Généreux,France,male,1980-12-01 00:00:00+00:00,2017-02-04 00:00:00+00:00
2,49,Havasy Kristof,Hungary,male,1998-06-13 00:00:00+00:00,2017-01-18 00:00:00+00:00
3,92,Honorata Nowak,Poland,female,1986-05-02 00:00:00+00:00,2017-09-21 00:00:00+00:00
4,113,Lucy Centeno Barrios,Spain,female,1970-11-03 00:00:00+00:00,2017-06-13 00:00:00+00:00
5,114,Canela Gaona Lozano,Spain,female,1997-04-01 00:00:00+00:00,2017-02-14 00:00:00+00:00


Avelaine Corbeil is one of the frequent customer from France

## 14. Movies with rating above average

In [3]:
SELECT title 
FROM movies.movies
WHERE movie_id IN
	(SELECT movie_id
	 FROM movies.renting
     GROUP BY movie_id
     HAVING AVG(rating) > 
		(SELECT AVG(rating)
		 FROM movies.renting));

,title
0,What Women Want
1,The Fellowship of the Ring
2,Harry Potter and the Philosopher's Stone
3,The Royal Tenenbaums
4,Waking Up in Reno
5,11'09''01 September 11
6,Harry Potter and the Chamber of Secrets
7,25th Hour
8,The Human Stain
9,Secondhand Lions


Movies 'What Women Want' and 'The Fellowship of the Ring' are having an average rating higher than the total average and also the most often rented movies.

## 15. Analyzing customer behaviour

In [1]:
-- Lets focus on customers who rented fewer than 5 movies.

SELECT *
FROM movies.customers as c
WHERE 5 > 
	(SELECT count(*)
	FROM movies.renting as r
	WHERE r.customer_id = c.customer_id);

,customer_id,name,country,gender,date_of_birth,date_account_start
0,2,Wolfgang Ackermann,Austria,male,1971-11-17 00:00:00+00:00,2018-10-15 00:00:00+00:00
1,3,Daniela Herzog,Austria,female,1974-08-07 00:00:00+00:00,2019-02-14 00:00:00+00:00
2,4,Julia Jung,Austria,female,1991-01-04 00:00:00+00:00,2017-11-22 00:00:00+00:00
3,5,Juliane Kirsch,Austria,female,1977-03-01 00:00:00+00:00,2018-12-16 00:00:00+00:00
4,6,Rowanne Couperus,Belgium,female,1994-04-05 00:00:00+00:00,2018-08-26 00:00:00+00:00
...,...,...,...,...,...,...
61,117,Moira Jurado Paez,Spain,female,1996-01-30 00:00:00+00:00,2018-11-09 00:00:00+00:00
62,119,Lisa Ibarra Valadez,Spain,female,1998-12-08 00:00:00+00:00,2019-03-06 00:00:00+00:00
63,120,Robin J. Himes,USA,male,1988-11-30 00:00:00+00:00,2018-08-06 00:00:00+00:00
64,122,Thelma B. Coward,USA,female,1971-04-15 00:00:00+00:00,2018-11-27 00:00:00+00:00


## 16. Customers who gave lower ratings

In [2]:
-- Let's identify customers who were not satisfied with movies they watched on MovieNow and report a list of customers with minimum rating smaller than 4.

SELECT *
FROM movies.customers AS c
WHERE 4 >  
	(SELECT MIN(rating)
	FROM movies.renting AS r
	WHERE r.customer_id = c.customer_id);


,customer_id,name,country,gender,date_of_birth,date_account_start
0,28,Sidney Généreux,France,male,1980-12-01 00:00:00+00:00,2017-02-04 00:00:00+00:00
1,41,Zara Mitchell,Great Britan,female,1994-07-08 00:00:00+00:00,2017-06-12 00:00:00+00:00
2,86,Albin Jaworski,Poland,male,1984-05-01 00:00:00+00:00,2017-12-15 00:00:00+00:00
3,120,Robin J. Himes,USA,male,1988-11-30 00:00:00+00:00,2018-08-06 00:00:00+00:00


Customers Sidney Généreux, Zara Mitchell, Albin Jaworski, and Robin J. Himes rated a movie with less than 4.

## 17. Most attention received movies on platform

In [3]:
-- report all movies with more than 5 ratings and all movies with an average rating higher than 8.

SELECT *
FROM movies.movies as m
WHERE 5 < 
	(SELECT COUNT(rating)
	FROM movies.renting as r
	WHERE r.movie_id = m.movie_id)
AND 8 <
	(SELECT AVG(rating)
	FROM movies.renting AS r
	WHERE r.movie_id = m.movie_id);

,movie_id,title,genre,runtime,year_of_release,renting_price
0,22,Imagining Argentina,Art House & International,107,2004,2.59
1,27,Monster,Drama,109,2004,2.09
2,40,Then She Found Me,Drama,100,2007,1.99
3,49,Harry Potter and the Half-Blood Prince,Science Fiction & Fantasy,153,2009,1.59
4,51,Morning Glory,Comedy,107,2010,2.99
5,54,Harry Potter and the Deathly Hallows – Part 1,Science Fiction & Fantasy,146,2010,2.79
6,57,Young Adult,Drama,94,2011,1.99
7,63,Django Unchained,Other,165,2012,2.69


We see Django Unchained is one of the movies that has more than 5 ratings with avg. rating higher than 8.

## 18. Customer with at least one rating

In [4]:
SELECT *
FROM movies.customers as c 
WHERE EXISTS
	(SELECT *
	FROM movies.renting AS r
	WHERE rating IS NOT NULL 
	AND r.customer_id = c.customer_id);

,customer_id,name,country,gender,date_of_birth,date_account_start
0,2,Wolfgang Ackermann,Austria,male,1971-11-17 00:00:00+00:00,2018-10-15 00:00:00+00:00
1,4,Julia Jung,Austria,female,1991-01-04 00:00:00+00:00,2017-11-22 00:00:00+00:00
2,7,Annelous Sneep,Belgium,female,1993-11-14 00:00:00+00:00,2018-05-12 00:00:00+00:00
3,8,Jaëla van den Dolder,Belgium,female,1990-08-31 00:00:00+00:00,2018-02-08 00:00:00+00:00
4,10,Arnout Veenhuis,Belgium,male,1984-07-26 00:00:00+00:00,2017-01-28 00:00:00+00:00
...,...,...,...,...,...,...
98,118,Digna Mateo Carrasquill,Spain,female,1978-11-01 00:00:00+00:00,2017-11-24 00:00:00+00:00
99,119,Lisa Ibarra Valadez,Spain,female,1998-12-08 00:00:00+00:00,2019-03-06 00:00:00+00:00
100,120,Robin J. Himes,USA,male,1988-11-30 00:00:00+00:00,2018-08-06 00:00:00+00:00
101,121,Annie R. Barnett,USA,female,1987-03-02 00:00:00+00:00,2017-09-13 00:00:00+00:00


Looks like there are 19 customers who haven't given any ratings.

## 19. Actors in comedies

In [7]:
-- list of actors who play in comedies and then, the number of actors for each nationality playing in comedies.

SELECT  a.nationality,COUNT(*)
FROM movies.actors AS a
WHERE EXISTS
	(SELECT ai.actor_id
	 FROM movies.actsin AS ai
	 LEFT JOIN movies.movies AS m
	 ON m.movie_id = ai.movie_id
	 WHERE m.genre = 'Comedy'
	 AND ai.actor_id = a.actor_id)
GROUP BY a.nationality;

,nationality,count
0,Northern Ireland,1
1,USA,22
2,South Africa,1
3,Canada,1
4,British,3


We see there is one actor each coming from South Africa, Canada and Northen Ireland, three actors from Great Britain, and 22 from the USA who played in a Comedy.

## 20. Young actors not coming from the USA

In [11]:
-- Identify actors who are not from the USA and actors who were born after 1990.

SELECT name, 
       nationality, 
       year_of_birth
FROM movies.actors
WHERE nationality <> 'USA'
INTERSECT 
SELECT name, 
       nationality, 
       year_of_birth
FROM movies.actors
WHERE year_of_birth > 1990;

,name,nationality,year_of_birth
0,Freddie Highmore,British,1992


Freddie Highmore as the only actor who is not from the USA and born after 1990.

## 21. Dramas with high ratings


In [12]:
-- The advertising team has a new focus. They want to draw the attention of the customers to dramas. So list of all movies that are in the drama genre and have an average rating higher than 9.

SELECT *
FROM movies.movies
WHERE movie_id IN 
   (SELECT movie_id
    FROM movies.movies
    WHERE genre = 'Drama'
    INTERSECT
    SELECT movie_id
    FROM movies.renting
    GROUP BY movie_id
    HAVING AVG(rating)>9);


,movie_id,title,genre,runtime,year_of_release,renting_price
0,42,No Country for Old Men,Drama,122,2007,1.49


'No Country for Old Men' is the only drama with average rating higher than 9.

# OLAP - Online Analytical Processing

## 1. Group of Customers (CUBE) VS ROLLUP

In [1]:
-- Table with the total number of male and female customers from each country.

SELECT gender, 
	   country,
	   COUNT(*)
FROM movies.customers
GROUP BY CUBE (country,gender)
ORDER BY country;

,gender,country,count
0,female,Austria,3
1,male,Austria,1
2,None,Austria,4
3,male,Belgium,3
4,None,Belgium,6
5,female,Belgium,3
6,None,Denmark,7
7,female,Denmark,4
8,male,Denmark,3
9,male,France,8


There are a total of 122 customers and that three out of four customers from Austria are female.

In [14]:
-- Look the total number of customers, the number of customers for each country, and the number of female and male customers for each country.

SELECT country,
       gender,
	   COUNT(*)
FROM movies.customers
GROUP BY  ROLLUP (country, gender)
ORDER BY country,gender;

,country,gender,count
0,Austria,female,3
1,Austria,male,1
2,Austria,None,4
3,Belgium,female,3
4,Belgium,male,3
5,Belgium,None,6
6,Denmark,female,4
7,Denmark,male,3
8,Denmark,None,7
9,France,female,5


USA and Austria have the least number of customers of 4.

## 2. Categories of movies

In [2]:
-- number of movies for different genres and release years.

SELECT genre,
       year_of_release,
       COUNT(*)
FROM movies.movies
GROUP BY CUBE(genre,year_of_release)
ORDER BY year_of_release;

,genre,year_of_release,count
0,None,2001.0,6
1,Drama,2001.0,2
2,Comedy,2001.0,2
3,Science Fiction & Fantasy,2001.0,2
4,Comedy,2002.0,3
...,...,...,...
57,Art House & International,NaN,1
58,Drama,NaN,36
59,Other,NaN,2
60,Animation,NaN,2


Only one movie from 2014 is available on MovieNow. The highest number of movies is from 2003 with 8 movies.

## 3. Analyzing average ratings

In [4]:
-- the average rating of movies across countries and genres.

SELECT 
	c.country, 
	m.genre, 
	AVG(r.rating) AS avg_rating 
FROM movies.renting AS r
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN movies.customers AS c
ON r.customer_id = c.customer_id
GROUP BY CUBE(c.country,m.genre); 

,country,genre,avg_rating
0,None,None,7.939024
1,France,Mystery & Suspense,6.000000
2,Slovenia,Action & Adventure,NaN
3,Spain,Animation,NaN
4,Poland,Comedy,8.200000
...,...,...,...
91,None,Art House & International,8.666667
92,None,Mystery & Suspense,6.833333
93,None,Comedy,7.766667
94,None,Action & Adventure,9.090909


The average over all records is 7.94.

## 4. Analyzing preferences of genres across countries

In [11]:
-- Evaluate the preferences of customers by averaging their ratings and counting the number of movies rented from each genre.

SELECT 
	c.country, 
	m.genre, 
	AVG(rating) AS avg_rating, 
	COUNT(*) AS num_rating
FROM movies.renting AS r
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN movies.customers AS c
ON r.customer_id = c.customer_id
GROUP BY ROLLUP (c.country,m.genre)
ORDER BY c.country, m.genre;

,country,genre,avg_rating,num_rating
0,Austria,Comedy,8.000000,1
1,Austria,Drama,6.000000,2
2,Austria,Mystery & Suspense,NaN,1
3,Austria,Science Fiction & Fantasy,6.666667,3
4,Austria,None,6.800000,7
...,...,...,...,...
83,None,Comedy,NaN,1
84,None,Drama,6.000000,2
85,None,Science Fiction & Fantasy,NaN,1
86,None,None,7.939024,578


Ther are total 578 ratings with Austria having least number of ratings of 7.

## 5. Exploring nationality and gender of actors

In [12]:
-- Count the number of actors in the table actors from each country, the number of male and female actors and the total number of actors.

SELECT 
	nationality ,
    gender ,
    COUNT(*)
FROM movies.actors
GROUP BY GROUPING SETS ((nationality), (gender), ()); 

,nationality,gender,count
0,None,None,145
1,Argentina,None,1
2,Spain,None,3
3,Italy,None,1
4,Puerto Rico,None,1
5,Iran,None,1
6,Northern Ireland,None,2
7,USA,None,91
8,Israel,None,1
9,Netherlands,None,1


There are 91 out of 145 actors are from the USA and that there are 90 male and 55 female actors.

## 6. Exploring rating by country and gender


In [13]:
SELECT 
	c.country, 
    c.gender,
	AVG(r.rating)
FROM movies.renting AS r
LEFT JOIN movies.customers AS c
ON r.customer_id = c.customer_id
GROUP BY GROUPING SETS ((country, gender), (country),(gender),());

,country,gender,avg
0,None,None,7.939024
1,Austria,male,6.000000
2,France,female,8.000000
3,Hungary,female,7.285714
4,None,None,8.000000
5,Spain,female,7.612903
6,Belgium,male,8.000000
7,USA,male,7.500000
8,Denmark,female,8.444444
9,Austria,female,7.000000


The overall average of ratings is 7.9.

# Business Case

MovieNow considers to invest money in new movies. Its generally more expensive to make movies available which were recently produced than older ones.

Check:

Do customers give better ratings to movies which were recently produced than older ones and is there any difference across countries.

In [16]:
SELECT c.country,
	m.year_of_release,
	COUNT(*) AS n_rentals,
	COUNT (DISTINCT r.movie_id) AS n_movies,
	AVG (rating) AS avg_rating
FROM movies.renting AS r
LEFT JOIN movies.customers AS c
ON c.customer_id = r.customer_id
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
	WHERE r.movie_id IN (
	SELECT movie_id
	FROM movies.renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 4)
AND r.date_renting >= '2018-04-01'
GROUP BY ROLLUP (m.year_of_release, c.country)
ORDER BY c.country, m.year_of_release;

,country,year_of_release,n_rentals,n_movies,avg_rating
0,Austria,2002.0,1,1,7.000000
1,Austria,2004.0,1,1,NaN
2,Austria,2007.0,1,1,6.000000
3,Austria,2011.0,1,1,6.000000
4,Belgium,2001.0,5,3,10.000000
...,...,...,...,...,...
126,None,2012.0,28,5,8.111111
127,None,2013.0,10,2,7.600000
128,None,2014.0,1,1,10.000000
129,None,2014.0,5,1,8.000000


We can see the total aggregation with 333 movie rentals, 50 different movies and an average rating of 7.9. 

Intuitively we can already see that there is no clear pattern of better ratings for movies with later year of release.

## Customer preference by gender

We just saw that customers have no clear preference for more recent movies over older ones. Now the management considers investing money in movies of the best rated genres.

In [17]:
SELECT genre,
	   AVG(rating) AS avg_rating,
	   COUNT(rating) AS n_rating,
       COUNT(*) AS n_rentals,     
	   COUNT(DISTINCT m.movie_id) AS n_movies 
FROM movies.renting AS r
LEFT JOIN movies.movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM movies.renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 3 )
AND r.date_renting >= '2018-01-01'
GROUP BY genre
ORDER BY avg_rating DESC; 

,genre,avg_rating,n_rating,n_rentals,n_movies
0,Action & Adventure,8.714286,7,9,2
1,Art House & International,8.500000,4,5,1
2,Other,8.428571,7,16,2
3,Science Fiction & Fantasy,8.276596,47,70,10
4,Comedy,7.950000,20,31,5
5,Animation,7.833333,6,10,2
6,Drama,7.748252,143,245,34
7,Mystery & Suspense,7.428571,7,19,3


Action & Adventure has the highest rating.

## Customer preference for actors

In [18]:
SELECT a.nationality,
       a.gender,
	   AVG(r.rating) AS avg_rating,
	   COUNT(r.rating) AS n_rating,
	   COUNT(*) AS n_rentals,
	   COUNT(DISTINCT a.actor_id) AS n_actors
FROM movies.renting AS r
LEFT JOIN movies.actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN movies.actors AS a
ON ai.actor_id = a.actor_id
WHERE r.movie_id IN ( 
	SELECT movie_id
	FROM movies.renting
	GROUP BY movie_id
	HAVING COUNT(rating) >= 4)
AND r.date_renting >= '2018-04-01'
GROUP BY CUBE (a.nationality,a.gender) 

,nationality,gender,avg_rating,n_rating,n_rentals,n_actors
0,Argentina,male,8.500000,4,5,1
1,Argentina,None,8.500000,4,5,1
2,Australia,female,8.666667,3,5,1
3,Australia,male,7.454545,11,17,3
4,Australia,None,7.714286,14,22,4
5,Austria,male,8.500000,2,6,1
6,Austria,None,8.500000,2,6,1
7,British,female,7.833333,54,78,3
8,British,male,8.105263,114,175,9
9,British,None,8.017857,168,253,12
